In [ ]:
%load_ext jupyter_black

# Pandas 

http://pandas.pydata.org/

Exploratory data analysis, предобработка данных, predictive modeling (в малой степени).
Проекции, слияние, фильтрация, группировка, агрегация, одним словом - **работа с таблицами.**

**Есть смысл читать документацию и думать о наибольшей эффективности выполнения тех или иных операций**.
Так, по-прежнему, если вы пишете цикл, вы что-то делаете не так; векторные операции for the win! И особенно это важно, если ваш код будет затем адаптироваться для [py]Spark. Благодаря DataFrame API Apache Spark, это делается просто.

Дополнительно с pandas можно ознакомиться тут:
1. [Более короткий](https://github.com/jvns/pandas-cookbook)
2. [Более обстоятельный](https://github.com/guipsamora/pandas_exercises/tree/master).

Если будете решать вторую — обязательно пройдите первые пару глав из первой, так как во второй тема индексаций пропущена полностью :)

## Data structures & types

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    data=[[1, 2, 3], [4, 5, 6]],
    columns=["A", "B", "C"],
    index=["X", "Y"],
)
df.head()

In [ ]:
df["D"] = ["a", "b"]
df

In [ ]:
# series -- "колонка" одного типа; можно относиться как к словарю из ИНДЕКСОВ в ЗНАЧЕНИЯ
s = pd.Series([1, 3, 5, np.nan, 6, 8], name="Some series")

print("Values types: ", s.dtype)
print("Series shape: ", s.shape)
print("Series indeix:", s.index)

# s.values
# s[2:4]
# s.index
# s

In [ ]:
s = pd.Series(
    [1, 3, 5, np.nan, 6, 8, 10],
    name="Some series",
    index=["q", "w", "e", "e", "r", "t", "y"],
)
s
# print("Index:", s.index)
# print(s[1:5])

# print(s["w":"e"])  # обратите внимание!
# 1) порядок -- лексикографический?
# 2) включён ли последний индекс?
# 3) а если сделать s["w":"z"]?

# print(s["e":"q"]) # обратите внимание!

In [ ]:
# А так ли все работает целочисленными индексами?
t = pd.Series(
    [1, 3, 5, np.nan, 6, 8, 10],
    name="Some series",
    index=[1, 2, 3, 4, 5, 6, 7],
)

# разница, на самом деле, заключена вот здесь, но об этом чуть позже
t.iloc[1:5], t.loc[1:5], s.iloc[1:5],  # s.loc[1:5]

В пандас есть бесчисленное количество методов для работы с табличными данными, и постоянно добавляется новое, поэтому есть смысл почитать к нему документацию, а не только чужой код. Может быть, так вам удастся избежать переизобретения колеса или велосипеда.

Тем, кто работает с реальными временными рядами, могут пригодиться вот такие штучки:

In [ ]:
# http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
dates = pd.date_range("20130101", periods=6)  # default frequency measure is 1 day

print(dates)

Потихоньку переходим к главному объекту пандаса: датафреймам.


In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
df = pd.DataFrame(
    data=np.random.randn(6, 4),
    index=dates,
    columns=list("ABCD"),
)

df

Ещё один способ задать датафрейм: ключи -- имена колонок, значения -- сами колонки

In [ ]:
df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20130102"),
        "C": pd.Series(1, index=list(range(4)), dtype="float32"),
        "D": np.array([3] * 4, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train"]),
        "F": "foo",
    }
)
df2

In [ ]:
df2.dtypes

## Accessing data
Как посмотреть на набор данных и что-то о нём понять

In [ ]:
df.head(3)  # , df.tail(2)

In [ ]:
print(df.index)
print(df.columns)
print(df.values)

df2.values

In [ ]:
# стандартные статистики по каждой колонке
df.describe()

In [ ]:
df.T

In [ ]:
df.T.index, df.index

In [ ]:
df.sort_index(axis=1, ascending=False), df

In [ ]:
df.sort_values(by="B")

## Выборка данных
#### NB!
While standard Python / Numpy expressions for selecting and setting are intuitive and come in handy for interactive work, for production code, we recommend the optimized pandas data access methods, .at, .iat, .loc, .iloc and .ix.

In [ ]:
df["A"]

In [ ]:
df[["A", "B"]]

In [ ]:
df.A

In [ ]:
df[:]  # по строкам

## Выборка по значениям индексов

In [ ]:
df["20130102":"20130104"]

In [ ]:
# первый индекс
print("First index: " + str(dates[0]))

print(df)

# выбираем всё по этому индексу
print(df.loc[dates[0]])

## Выборка по порядковым номерам индексов

In [ ]:
df.iloc[3:5]

In [ ]:
# доступ до отдельного значения
print(df.iat[1, 1])
print(df.iloc[1, 1])  # то же

In [ ]:
# выборка по значениям в колонках [ничего не напоминает?]
df[df.A > 0]

In [ ]:
# фильтрация
df2 = df.copy()
df2["E"] = ["one", "one", "two", "three", "four", "three"]

# то же, что filter по вхождению
# df2[]
df2[df2["E"].isin(["hello", "woow", "three", "one"])]
# df2['E'].isin(['hello', 'woow', 'three', 'one'])

## Обновление

In [ ]:
s1 = pd.Series([1, 2, 3, 4, 5, 6], index=pd.date_range("20130102", periods=6))
df["F"] = s1
df

In [ ]:
# выборка по индексам
df.at["2013-01-04", "A"] = 99.0
df.iat[0, 1] = 999999
df.loc[:, "D"] = np.array([5] * len(df))
df

In [ ]:
# выборка по условию
df[df < -0.5] = 0 * df
df
# df > 0

In [ ]:
# df[uslovie(df.ix)] = func(df)

In [ ]:
df["cat_variable"] = ["Barsik", "Barsik", "Marusya", "Barsik", "Garfield", "Marusya"]

In [ ]:
df

In [ ]:
cats_df = pd.get_dummies(df["cat_variable"], prefix="cat")
cats_df

In [ ]:
vals = pd.concat([df.drop(["F", "A", "cat_variable"], axis=1), cats_df], axis=1)
vals

In [ ]:
df.drop(["F", "A", "cat_variable"], axis=1)

На самом деле, в pandas есть довольно продвинутый sql-like синтаксис, но нам вряд ли он понадобится.

In [ ]:
# sample data
orders_data = {
    "customer_id": [1, 2, 3, 1, 2, 3, 4],
    "order_amount": [100, 200, 150, 300, 120, 50, 500],
}
orders = pd.DataFrame(orders_data)

customers_data = {
    "customer_id": [1, 2, 3, 4],
    "customer_name": ["Alice", "Bob", "Charlie", "David"],
    "city": ["New York", "Los Angeles", "New York", "Chicago"],
}
customers = pd.DataFrame(customers_data)

In [ ]:
# merge orders with customers on customer_id
merged_df = pd.merge(orders, customers, on="customer_id")
merged_df

In [ ]:
# Group by city and aggregate the total revenue per city
city_revenue = (
    merged_df.groupby("city")["order_amount"].sum().reset_index(name="total_revenue")
)
city_revenue

In [ ]:
# apply a custom function to find the top customer (by total order amount) in each city
def top_customer_in_city(df):
    top_customer = df.groupby("customer_name")["order_amount"].sum().idxmax()
    return top_customer


# group by city and apply the custom function to find the top customer in each city
top_customers = (
    merged_df.groupby("city")
    .apply(top_customer_in_city)
    .reset_index(name="top_customer")
)
top_customers

---
## Упражнения
*Не являются домашкой

In [ ]:
# df = pd.read_csv("data/estonia-passenger-list.csv") 
## если видеть не можете "Титаник": 
## https://www.kaggle.com/christianlillelund/passenger-list-for-the-estonia-ferry-disaster
## Задание 0
## поисследуйте распределения значений фич -- 
# print(df["Country"].value_counts().head(), "\n")
## Что делает эта цепочка вызовов? Какой объект после каждого? Что показывает?
# print(df.isna().sum())
# df.head()

In [ ]:
train.head(2)

In [ ]:
# pd.get_dummies(train["Pclass"])
# pd.get_dummies(train["Pclass"], prefix="Pclass")

In [ ]:
# Задание 2
# Написать код, заполняющий пропуски в "численных" колонках 
# 2a -- значением (-1)
# 2b -- средним значением по колонке

In [ ]:
# Задание 3
# Понять, какие фичи -- номинальные (категориальные) -- без осмысленного порядка над ними. 
# Применить к ним пандасовский dummy_encoding 

In [ ]:
# Задание 4
# Выделить Survived как отдельный вектор (или Series)

In [ ]:
# Задание 5
# Нормализуйте средствами pandas (нельзя использовать sklearn) отдельно 
# каждую колонку -- с возрастом и со стоимостью билета
# 5a. min-max scaling
# 5b. вычесть среднее и разделить на стандартное отклонение

In [ ]:
# Задание 6
# Постройте
# train.pivot_table('PassengerId', 'Pclass', 'Survived', 'count').plot(kind='bar', stacked=True)
# Погуглите, что это? О чём нам говорит этот график?

In [ ]:
# Задание 7
# Для каждой фичи, кажущейся вам полезной, постройте гистограмму с помощью pandas-hist

**P.S.** Задачи, похожие на эти упражнения, в мире машинного обучения и статистики называются Exploratory Data Analysis (EDA). Наш курс смещен в сторону теории обучения, и заниматься подобным мы больше не будем. Тем, кому надо, советую пройти курсы по feature engineering, data visualization, & data cleaning в Kaggle Learn.

---
This notebook is basen on Anton Alekseev's practical lessons at MCS SPBU and pandas documentation.